In [1]:
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam,SGD
import random


In [2]:
data = pd.read_csv(r'D:\serevice_chatBOt\med-sqd\medquad.csv\medquad.csv')
data=data.drop(['source'],axis=1)
data.isnull().sum()
data = data.dropna()
words = []
classes = []
documents = []
ignoreLetters = ["?", "!", ".", ","]
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
data.head()

,question,answer,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,Glaucoma


In [4]:
for index,row in data.iterrows():
    wordList = nltk.word_tokenize(row['question'])
    words.extend(wordList)
    documents.append((wordList, row['answer']))
    if row['answer'] not in classes:
        classes.append(row['answer'])

In [5]:

lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

In [6]:
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append([bag, outputRow])

trainX = np.array([item[0] for item in training], dtype=np.float32)
trainY = np.array([item[1] for item in training], dtype=np.float32)

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(trainX[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(trainY[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Use SGD optimizer with parameters
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
model.fit(trainX, trainY, epochs=100, batch_size=5, verbose=1) # Adjust epochs and batch size as needed

# Save the model
model.save('medquad_model.h5')

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
3279/3279 ━━━━━━━━━━━━━━━━━━━━ 79s 23ms/step - accuracy: 0.0203 - loss: 9.6016
Epoch 2/200
 425/3279 ━━━━━━━━━━━━━━━━━━━━ 1:11 25ms/step - accuracy: 0.0125 - loss: 9.5320